# Network Verification using DNNV

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import required libraries

In [ ]:
import torch
import torch.onnx
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import os, sys
import cv2


### Install DNNV + dependencies + create virtual environment

### Run the following command lines
sudo apt-get install python3-venv

sudo apt-get install python3.7-venv


python3.7 -m venv .venv

. .venv/bin/activate

pip install --upgrade pip 

pip install flit

flit install -s


pip install dnnv

pip install onnx

pip install numpy==1.16.0


git clone https://github.com/dlshriver/DNNV.git

cd DNNV

./manage.sh init

. .env.d/openenv.sh

./manage.sh install reluplex

./manage.sh install eran 


Other verifiers can be installed:planet mipverify neurify plnn marabou nnenum verinet

### Convert models to ONNX

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.bn1 = nn.BatchNorm2d(12)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.bn2 = nn.BatchNorm2d(24)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(24*53*53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1,24*53*53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net1 = Net()
state_dict1 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/best_model1.pth')
net1.load_state_dict(state_dict1)
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(net1, dummy_input, "/content/drive/MyDrive/KASHIKO/MODELS/best_model1.onnx")

In [ ]:
net2 = Net()
state_dict2 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/best_model2.pth')
net2.load_state_dict(state_dict2)
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(net2, dummy_input, "/content/drive/MyDrive/KASHIKO/MODELS/best_model2.onnx")

In [ ]:
net3 = Net()
state_dict3 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/best_model3.pth')
net3.load_state_dict(state_dict3)
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(net3, dummy_input, "/content/drive/MyDrive/KASHIKO/MODELS/best_model3.onnx")

### Create property file and save to Drive (manually)

### Template of property file (.py)
from dnnv.properties import *
import numpy as np

N = Network("N")
x = Image("{input_path}")

epsilon = Parameter("epsilon", type=float, default=(2.0 / 255))
true_class = {true_class}

Forall(
    x_,
    Implies(
        ((x - epsilon) < x_ < (x + epsilon)) & (0 < x_ < 1),
        argmax(N(x_)) == true_class,
    ),
)

### Convert image to .npy

In [ ]:
im = Image.open('/content/drive/MyDrive/KASHIKO/DATASET/VERIF/holdingpoint.jpg').convert("RGB")
im = np.transpose(im,(2, 0, 1))
im = np.array(im)
np.save("/content/drive/MyDrive/KASHIKO/DATASET/VERIF/holdingpoint.npy",im)

In [ ]:
im = Image.open('/content/drive/MyDrive/KASHIKO/DATASET/VERIF/noholdingpoint.jpg').convert("RGB")
im = np.transpose(im,(2, 0, 1))
im = np.array(im)
np.save("/content/drive/MyDrive/KASHIKO/DATASET/VERIF/noholdingpoint.npy",im)

### Launch analysis

### Run the following command lines
python3 -m dnnv --reluplex --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model1.onnx /content/drive/MyDrive/KASHIKO/property_holdingpoint.py

python3 -m dnnv --eran --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model1.onnx /content/drive/MyDrive/KASHIKO/property_holdingpoint.py

python3 -m dnnv --reluplex --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model1.onnx /content/drive/MyDrive/KASHIKO/property_noholdingpoint.py

python3 -m dnnv --eran --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model1.onnx /content/drive/MyDrive/KASHIKO/property_noholdingpoint.py

### Run the following command lines
python3 -m dnnv --reluplex --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model2.onnx /content/drive/MyDrive/KASHIKO/property_holdingpoint.py

python3 -m dnnv --eran --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model2.onnx.onnx /content/drive/MyDrive/KASHIKO/property_holdingpoint.py

python3 -m dnnv --reluplex --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model2.onnx /content/drive/MyDrive/KASHIKO/property_noholdingpoint.py

python3 -m dnnv --eran --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model2.onnx /content/drive/MyDrive/KASHIKO/property_noholdingpoint.py

### Run the following command lines
python3 -m dnnv --reluplex --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model3.onnx /content/drive/MyDrive/KASHIKO/property_holdingpoint.py

python3 -m dnnv --eran --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model3.onnx.onnx /content/drive/MyDrive/KASHIKO/property_holdingpoint.py

python3 -m dnnv --reluplex --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model3.onnx /content/drive/MyDrive/KASHIKO/property_noholdingpoint.py

python3 -m dnnv --eran --network N /content/drive/MyDrive/KASHIKO/MODELS/best_model3.onnx /content/drive/MyDrive/KASHIKO/property_noholdingpoint.py